In [5]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-03-27 16:26:08--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  4.44MB/s    in 0.2s    

2023-03-27 16:26:10 (4.44 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [32]:
import torch
from torch import nn
from torch.nn import functional as F


In [46]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [47]:
print(f'length of dataset in charcters: {len(text)}')

length of dataset in charcters: 1115394


In [48]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [49]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [50]:
# Tokenize input Text (Character level tokenizer)
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # takes string, and outputs a list of integer
decode = lambda l: ''.join([itos[i] for i in l])# takes list of integer, and outputs a string

print(encode('hello, there!'))
print(decode(encode('hello, there!')))

[46, 43, 50, 50, 53, 6, 1, 58, 46, 43, 56, 43, 2]
hello, there!


In [51]:
import torch

# convert text encoding into torch.tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:100])

torch.Size([1115394]) <built-in method type of Tensor object at 0x127388270>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [52]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [53]:
train_data.shape, val_data.shape

(torch.Size([1003854]), torch.Size([111540]))

In [54]:
# set size of input to transformer
block_size = 8
train_data[:block_size+1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

- The block of input above represents 8 different context examples. For example, if the word 18 appears, 47 appears next. If 18, 47, 56, and 58 appears consecutively, 1 will come next, ane etc.
  

In [55]:
x = train_data[:block_size]
y = train_data[1: block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target is {target}')

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


## Batch Dimension
- Mini Batch helps with efficiency
- process multiple chunks at the same time

In [56]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for prediction?

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data)-batch_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')

print(f'''input Tensor info
shape: {xb.shape},
{xb}
''')
print(f'''target Tensor info
shape: {yb.shape},
{yb}''')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is {context.tolist()} the target is {target}')

input Tensor info
shape: torch.Size([4, 8]),
tensor([[ 1, 60, 39, 47, 50,  1, 63, 53],
        [46, 43, 39, 60, 43, 52,  1, 44],
        [ 1, 46, 43, 56, 43,  1, 63, 53],
        [61, 47, 50, 50,  1, 57, 39, 63]])

target Tensor info
shape: torch.Size([4, 8]),
tensor([[60, 39, 47, 50,  1, 63, 53, 59],
        [43, 39, 60, 43, 52,  1, 44, 53],
        [46, 43, 56, 43,  1, 63, 53, 59],
        [47, 50, 50,  1, 57, 39, 63,  0]])
when input is [1] the target is 60
when input is [1, 60] the target is 39
when input is [1, 60, 39] the target is 47
when input is [1, 60, 39, 47] the target is 50
when input is [1, 60, 39, 47, 50] the target is 1
when input is [1, 60, 39, 47, 50, 1] the target is 63
when input is [1, 60, 39, 47, 50, 1, 63] the target is 53
when input is [1, 60, 39, 47, 50, 1, 63, 53] the target is 59
when input is [46] the target is 43
when input is [46, 43] the target is 39
when input is [46, 43, 39] the target is 60
when input is [46, 43, 39, 60] the target is 43
when input is 

In [57]:
## Simplest Language Model ( bigram language model )
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for next tokn from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            # reshape logits
            B, T, C =  logits.shape
            logits = logits.view(B*T, C)
            targets  = targets.view(B*T)
            # negative log loss
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indicies in current context
        for _ in range(max_new_tokens):
            # get prediction
            logits, loss = self(idx) # goes to def forward() function
            # focus only on last time step
            logits = logits[:, -1, :] # shape: (B. C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss)


decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist())


torch.Size([32, 65]) tensor(4.5629, grad_fn=<NllLossBackward0>)


"\nlfJeukRuaRJKXAYtXzfJ:HEPiu--sDioi;ILCo3pHNTmDwJsfheKRxZCFs\nlZJ XQc?:s:HEzEnXalEPklcPU cL'DpdLCafBheH"

In [18]:
# Create pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # usually 1e-4 works well in bigger models

In [37]:
batch_size = 32

for steps in range(10000):
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.440112590789795


In [38]:
print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist())
)


Cosp gr o n g co,
LORire tewe hine thaillo ethal tat ckn mbenthereld Myonoupayod moth h ind lilaven hanere, iokiverse my doupere, feary, 'st mey?
IO:
Bu ith, ly' isped stu m pr
pl l INENoforothe I n:
Hen!owaimy ayshe 'Bern ke
Winghed t he Thivererenyomizeyout,
ICENE:
TYou yor itrthe so nd pan b, thed henilat?
Ay ue hever terereadund s t,
Wealeyons t t
Sor putingheie'HENUSels cong f fons y tso achibes ay d f is!
ralencindlal thad, Me s hero hin!


D:
IUSppl t; I mad, m.
Yoreves imy wicheri? hall.


## Self Attention

### Mathematical Trick in self-attention

- Let's make tokens talk to each other while training with self-attention algorithm
- a certain token should not communicate with future token but with previous tokens.
  - for example, token #5 in sentence should not communicate with token #6, 7 and so on because those are the tokens that we need to predict
  - Instead, the token #5 should communicate with token # 1, 2, 3, and 4, 

- Then, what is the easiest way that tokens can communicate?
  - maybe average all previous tokens?
    - very lossy
    - 

In [33]:
import torch
# Toy example
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape, x

(torch.Size([4, 8, 2]),
 tensor([[[ 0.1808, -0.0700],
          [-0.3596, -0.9152],
          [ 0.6258,  0.0255],
          [ 0.9545,  0.0643],
          [ 0.3612,  1.1679],
          [-1.3499, -0.5102],
          [ 0.2360, -0.2398],
          [-0.9211,  1.5433]],
 
         [[ 1.3488, -0.1396],
          [ 0.2858,  0.9651],
          [-2.0371,  0.4931],
          [ 1.4870,  0.5910],
          [ 0.1260, -1.5627],
          [-1.1601, -0.3348],
          [ 0.4478, -0.8016],
          [ 1.5236,  2.5086]],
 
         [[-0.6631, -0.2513],
          [ 1.0101,  0.1215],
          [ 0.1584,  1.1340],
          [-1.1539, -0.2984],
          [-0.5075, -0.9239],
          [ 0.5467, -1.4948],
          [-1.2057,  0.5718],
          [-0.5974, -0.6937]],
 
         [[ 1.6455, -0.8030],
          [ 1.3514, -0.2759],
          [-1.5108,  2.1048],
          [ 2.7630, -1.7465],
          [ 1.4516, -1.5103],
          [ 0.8212, -0.2115],
          [ 0.7789,  1.5333],
          [ 1.6097, -0.4032]]]))

In [34]:
# we want x[b, t] = mean_{i<=t} x[b, i]
xbow = torch.zeros((B, T, C))#x bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

x[0], xbow[0]

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [35]:
# check if calc is correct
x_avg_3 = (x[0][0] + x[0][1] + x[0][2]) / 3
x_avg_3, xbow[0][2]


(tensor([ 0.1490, -0.3199]), tensor([ 0.1490, -0.3199]))

In [36]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [37]:
# we can make calculation above more efficient with matrix calculation
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print(a)
print(b)
print(c)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [38]:
# with triangular ones matrix, we can efficiently calculate cumulative sum of certain matrix
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print(a)
print(b)
print(c)

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [39]:
# If we normalize each row of triangular matrix (sum of all element in row = 1), 
# we can calcualte cumulative average
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [40]:
# lets vectorize the first problem (Version 2)
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [41]:
xbow2 = wei @ x # (T, T) @ (B, T, C) ==> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [42]:
xbow[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [43]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [44]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [45]:
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[1.0000,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.5000, 0.5000,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.3333, 0.3333, 0.3333,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.2500, 0.2500, 0.2500, 0.2500,   -inf,   -inf,   -inf,   -inf],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000,   -inf,   -inf,   -inf],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667,   -inf,   -inf],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429,   -inf],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [46]:
wei = F.softmax(wei, dim=-1) # normailzation operation
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [47]:
# Version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

- In version 3
  - torch.zeros matrix shows the strength of affinity of each tokens
    - these affinity will be data dependent
    - depending on values, the relationship(affinity) is formed by training
  - masked_fill part represents that all future token will not be used in calculating affinity of tokens
- you can do weignted aggregation of past elements by matrxi mul of lowe triangular fashion

In [51]:
# Version 4: SELF ATTENTION!
# the weight does 
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # Batch, time, channels
x = torch.randn(B, T, C)
# every single token at each position will emit two vectors: Querey and Key
## Query : what am I looking for?
## Key : What do I contain?
## Affinity is calculated by dot prod of Key and Query

# single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

# produce key and querey by forwarding x on key, query module
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16) (headsize)

# for every batch, we will have TXT(Affinity)
# Now, weight are DATA-DEPENDENT!
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) -> (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)

# propagate x on value
# value gives private identity to each token in x
v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [52]:
wei[0]

tensor([[0.0248, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0052, 0.0091, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0521, 0.0135, 0.2482, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3171, 0.0214, 0.1642, 0.1188, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0412, 0.0487, 0.1046, 0.0742, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1060, 0.5347, 0.2059, 0.1030, 0.7402, 0.0192, 0.0000, 0.0000],
        [0.4298, 0.3409, 0.1769, 0.2027, 0.0480, 0.8472, 0.2329, 0.0000],
        [0.0238, 0.0316, 0.1002, 0.5013, 0.0117, 0.1336, 0.7671, 1.0000]],
       grad_fn=<SelectBackward0>)

In [50]:
v

tensor([[[-1.5713e-01,  8.8009e-01,  1.6152e-01, -7.8239e-01, -1.4289e-01,
           7.4676e-01,  1.0068e-01, -5.2395e-01, -8.8726e-01,  1.9067e-01,
           1.7616e-01, -5.9426e-01, -4.8124e-01, -4.8599e-01,  2.8623e-01,
           5.7099e-01],
         [ 8.3212e-01, -8.1437e-01, -3.2425e-01,  5.1913e-01, -1.2520e-01,
          -4.8982e-01, -5.2867e-01, -3.1393e-02,  1.0723e-01,  8.2692e-01,
           8.1322e-01, -2.7132e-02,  4.7754e-01,  4.9801e-01, -1.3769e-01,
           1.4025e+00],
         [ 6.0346e-01, -2.4995e-01, -6.1588e-01,  4.0678e-01,  3.3283e-01,
          -3.9097e-01,  1.3119e-01,  2.1718e-01, -1.2991e-01, -8.8281e-01,
           1.7242e-01,  4.6522e-01, -4.2710e-01, -7.6754e-02, -2.8524e-01,
           1.3875e+00],
         [ 6.6568e-01, -7.0960e-01, -6.0986e-01,  4.3484e-01,  8.9754e-01,
          -9.2983e-01,  6.8324e-02,  1.8632e-01,  5.4002e-01,  2.4271e-01,
          -6.9225e-01,  4.9774e-01,  4.8503e-01,  6.6076e-01,  8.7671e-01,
           7.4567e-02],
    

## Attention is a communication mechanism
- Think attention as a Directed Graph structure.
- 
- every node has vector of info 
- Attention doesn't have notion of space

## Each Batch doesn't communicate with each other

## Difference between encoder and decoder
- In encoder, you need to delete the trill part so that all of the tokens can communictate with each other. However, in decoder, tril part is ommited, since 

## Self-Attention vs Cross-Attention
- In self attention, all of the K, Q, and V comes from the same source. (x). However, in Cross-attention, seperate source of nodes that produces keys and values.



In [57]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
# we normalize the weights with sqrt(headsize) since
# too extreme value can converge softmax func to 
# one hot encoding vector.
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [58]:
k.var(), q.var(), wei.var()

(tensor(1.0037), tensor(1.0966), tensor(1.2769))

In [61]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [62]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*100, dim=-1)

tensor([4.2484e-18, 3.9754e-31, 2.0612e-09, 3.9754e-31, 1.0000e+00])